In [0]:
%load_ext autoreload
%autoreload 2

import sys
from config.table_config import PROJECT_PATH

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

from config import *
from utils import BronzeIngestion
from pyspark.sql.types import *
from pyspark.sql.functions import col, year, month

configure_spark(spark)

In [0]:
orders_new_schema = StructType([
    StructField("order_id",       IntegerType(), nullable=True),  # Integer not UUID
    StructField("customer_id",    IntegerType(), nullable=True),
    StructField("seller_id",      IntegerType(), nullable=True),
    StructField("order_date",     DateType(),    nullable=True),  # Date not Timestamp
    StructField("payment_method", StringType(),  nullable=True)
    # Note: No order_total_price in orders_new
])

In [0]:
ingestion = BronzeIngestion(
    spark=spark,
    table_display_name="ORDERS_NEW",
    source_path=BRONZE_ORDERS_NEW,
    target_table=BRONZE_ORDERS_NEW_TABLE
)

ingestion.check_source(dbutils)
ingestion.read(schema=orders_new_schema)

# Add partition columns
ingestion.df = ingestion.df \
    .withColumn("order_year",  year(col("order_date"))) \
    .withColumn("order_month", month(col("order_date")))

ingestion.write_iceberg(
    catalog_name=CATALOG_NAME,
    bronze_schema=BRONZE_SCHEMA,
    partition_cols=["order_year", "order_month"]
)

ingestion.show_table_details()